# Project: Investigate The Movie Database (TMDb) Dataset

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

[test](#conclusions)

<a id='intro'></a>
## Introduction

> The Movie Database (TMDb) dataset, originally from Kaggle was cleaned and provided by Udacity. The dataset contains information about 10,000 movies from the year 1960 to 2015 collected from The Movie Database (TMDb), including user ratings and revenue.
The questions that I will explore over the course of the report:
- Q1: What kind of properties are associated with movies that have high revenues?
- Q2: What genres are most popular from year to year? / How have movie genres changed over time?
- Q3: How have movies based on novels performed relative to movies not based on novels?
- Q4: How do the attributes differ between Universal Pictures and Paramount Pictures?

In [6]:
# import statements for all of the packages that I plan to use. 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

> In this section of the report, I will load in the data, check for cleanliness, and then trim and clean the dataset for analysis. 

### General Properties
Below are the questions to answer using pandas to explore ```tmdb-movies.csv``` and have a holistic understanding of the data set:

- Number of samples & columns in the dataset
- Duplicate rows in the dataset
- Datatypes of the columns
- Features with missing values
- Number of non-null unique values for features in the dataset
- What those unique values are and counts for each
- Number of rows with missing values in the dataset
- Descriptive statistics for the dataset

In [3]:
# Load the data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
movie_df = pd.read_csv('tmdb-movies.csv')
movie_df.head(3)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08


>**Observations:** 
>- Columns like ```cast```, ```keywords```, ```genres``` and ```production_companies```, contain multiple values separated by pipe (|) characters. 
- The final two columns ending with “_adj” show the budget and revenue of the associated movie in terms of 2010 dollars, accounting for inflation over time. I will drop the original value and retain the two columns so that I can better compare dollar values from one year to another.
- Noticed that the ```vote_count``` is different for all the movies, so I will not be able to calculate the popularity of the movies based on the value.

#### Number of samples & columns in the dataset
Based on the result shown below, there are a total of 10,866 number of movies and 21 columns in the dataset.

In [8]:
# To check the size of the dataset
movie_df.shape

(10866, 21)

#### Duplicate rows in the dataset
There is only 1 duplicated row which I will drop it in the next section on data cleaning.

In [9]:
# To get the total number of duplicated rows in the dataset 
movie_df.duplicated().sum()

1

In [10]:
# To check and make sure that the duplicated row is exactly the same
movie_df[movie_df.duplicated(keep=False)]

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
2089,42194,tt0411951,0.59643,30000000,967000,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,NaN,Dwight H. Little,Survival is no game,...,"In the year of 2039, after World Wars destroy ...",92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,3/20/10,110,5.0,2010,30000000.0,967000.0
2090,42194,tt0411951,0.59643,30000000,967000,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,NaN,Dwight H. Little,Survival is no game,...,"In the year of 2039, after World Wars destroy ...",92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,3/20/10,110,5.0,2010,30000000.0,967000.0


#### Datatype of column in the dataset
```info``` is use to find the basic information like datatype of column in the dataset. Noticed that column such as ```release_date``` is not using the date datatype, ```budget_adj``` and ```revenue_adj``` are float datatype which I will work on it in the next section.

In [7]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
id                      10866 non-null int64
imdb_id                 10856 non-null object
popularity              10866 non-null float64
budget                  10866 non-null int64
revenue                 10866 non-null int64
original_title          10866 non-null object
cast                    10790 non-null object
homepage                2936 non-null object
director                10822 non-null object
tagline                 8042 non-null object
keywords                9373 non-null object
overview                10862 non-null object
runtime                 10866 non-null int64
genres                  10843 non-null object
production_companies    9836 non-null object
release_date            10866 non-null object
vote_count              10866 non-null int64
vote_average            10866 non-null float64
release_year            10866 non-null int64
budget_adj              1

#### Features with missing values in the dataset
There are a number of null values in the dataset for these columns: ```imdb_id```, ```cast```, ```homepage```, ```director```, ```tagline```, ```keywords```, ```overview```, ```genres``` and ```production_companies```. Huge number of null values for ```homepage```, ```tagline```, ```keywords``` and ```production_companies``` but since some of them are not necessary for answering my above questions, I will drop them during the data cleaning section.

In [8]:
movie_df.isnull().sum()

id                         0
imdb_id                   10
popularity                 0
budget                     0
revenue                    0
original_title             0
cast                      76
homepage                7930
director                  44
tagline                 2824
keywords                1493
overview                   4
runtime                    0
genres                    23
production_companies    1030
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj                 0
revenue_adj                0
dtype: int64

#### Number of unique values for the dataset
To see the total number of unique values for each column.

In [9]:
movie_df.nunique()

id                      10865
imdb_id                 10855
popularity              10814
budget                    557
revenue                  4702
original_title          10571
cast                    10719
homepage                 2896
director                 5067
tagline                  7997
keywords                 8804
overview                10847
runtime                   247
genres                   2039
production_companies     7445
release_date             5909
vote_count               1289
vote_average               72
release_year               56
budget_adj               2614
revenue_adj              4840
dtype: int64

#### Number of rows with missing values in the dataset
To check the total number of rows with at least one column with missing values. 

In [10]:
movie_df.isnull().any(axis=1).sum()

8874

#### Descriptive statistics for the dataset
Based on the table shown below, 50% of the value in ```budget``` and ```revenue``` are of zero value. There are also a number of zero value in ```runtime```.

In [11]:
movie_df.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,1.086600e+04,1.086600e+04,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,66064.177434,0.646441,1.462570e+07,3.982332e+07,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,1.000185,3.091321e+07,1.170035e+08,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,0.000065,0.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,0.207583,0.000000e+00,0.000000e+00,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,0.383856,0.000000e+00,0.000000e+00,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,0.713817,1.500000e+07,2.400000e+07,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning
After the above discussion on the structure of the dataset and the problems that need to be cleaned, the following are the cleaning steps to be performed:

1. Drop Extraneous Columns
2. Drop the Duplicates
3. Convert ``` release_date``` column to date datatype
4. Convert ```budget_adj``` and ```revenue_adj``` to integer datatype
5. Drop rows with Missing Values
6. Split values for   ```cast```, ```keywords```, ```genres``` and ```production_companies``` 

#### Drop Extraneous Columns
- Drop columns that aren't relevant to our questions. Columns to Drop: ```id```, ```imdb_id```, ```popularity```, ```homepage```, ```overview```, ```tagline```
- ```budget``` and ```revenue``` will also be drop as I will be using the final two columns ending with “_adj” which show the budget and revenue of the associated movie in terms of 2010 dollars, accounting for inflation over time.

In [12]:
# drop columns from the dataset
movie_df.drop(['imdb_id', 'popularity', 'homepage', 'overview', 'tagline'], axis=1, inplace=True)

# confirm changes
movie_df.head(1)

,id,budget,revenue,original_title,cast,director,keywords,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,monster|dna|tyrannosaurus rex|velociraptor|island,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09


#### Drop the Duplicates

In [12]:
# drop duplicates in the datasets
movie_df.drop_duplicates(inplace=True)

# print number of duplicates again to confirm dedupe - should be 0
movie_df.duplicated().sum() 

0

#### Convert ``` release_date``` column to Date datatype
To convert ```release_date``` column to date format (year-month-day).

In [13]:
movie_df['release_date'] = movie_df['release_date'].astype(date)

NameError: name 'date' is not defined

#### Convert ``` budget_adj``` and ```revenue_adj``` column to Integer datatype

In [17]:
# convert budget_adj and revenue_adj columns to integer
change_datatype = ['budget_adj', 'revenue_adj']

movie_df[change_datatype] = movie_df[change_datatype].astype(int)
# To confirm the datatypes have been changed
movie_df.dtypes

id                        int64
imdb_id                  object
popularity              float64
budget                    int64
revenue                   int64
original_title           object
cast                     object
homepage                 object
director                 object
tagline                  object
keywords                 object
overview                 object
runtime                   int64
genres                   object
production_companies     object
release_date             object
vote_count                int64
vote_average            float64
release_year              int64
budget_adj                int32
revenue_adj               int32
dtype: object

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [4]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [5]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!

In [16]:
test = movie_df['genres'].str.split('|')                        
print(test)                         

0           [Action, Adventure, Science Fiction, Thriller]
1           [Action, Adventure, Science Fiction, Thriller]
2                   [Adventure, Science Fiction, Thriller]
3            [Action, Adventure, Science Fiction, Fantasy]
4                                [Action, Crime, Thriller]
5                    [Western, Drama, Adventure, Thriller]
6           [Science Fiction, Action, Thriller, Adventure]
7                      [Drama, Adventure, Science Fiction]
8                   [Family, Animation, Adventure, Comedy]
9                              [Comedy, Animation, Family]
10                              [Action, Adventure, Crime]
11           [Science Fiction, Fantasy, Action, Adventure]
12                                [Drama, Science Fiction]
13                       [Action, Comedy, Science Fiction]
14                    [Action, Adventure, Science Fiction]
15                        [Crime, Drama, Mystery, Western]
16                               [Crime, Action, Thrille

In [18]:
test[1][1]

'Adventure'